<div style="text-align: center;background-color: gold; padding: 25px; border-radius: 5px; margin-bottom: 10px;">
    <h2> BOX ID PREDICTION USING MACHINE LEARNING AND Statistical Approach.</h2>
</div>

* Main Idea : `Robi Datathon 3.0 Problem 2` 😊✨
* Coding    : `Tanbeer Jubaer` 🌟 and `Minhazul Islam` 🏆

# Dataset Description
For this competition,

The MAGIC_KEY id is the unique identifier and the rest columns are the features.

Magic Keys given in “problem 2.csv” purchased only one box of milk and/or meat in the first 15 days of March-2019. You need to predict which boxes (There are a total of 290 boxes, with box IDs ranging from 1 to 290) were purchased by these customers in this period. Follow "sample submission 2.csv" as the submission template.

## Files

    boxes.csv - contains box informations
    purchase.csv - contains a product purchase info
    problem 2.csv - the test set
    sample submission 2.csv - a sample submission file in the correct format

<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3> <a href="https://github.com/oitik/Robi-Datathon-3.0/blob/master/robi-datathon-p2-2.ipynb">Click --> Alternate Notebook with ML Approach</a></h3>
    
</div>


<div style="text-align: center;">
    <h2> 1. Import Necessary Libraries</h2>
</div>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

%matplotlib inline

from tqdm import tqdm
import string
import statistics
import re
import joblib
import random
import statistics
from collections import Counter
from scipy import stats
import os

# import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
from wordcloud import WordCloud

## Import Data

In [10]:
box=pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-2/boxes.csv')
purchase=pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-2/purchase.csv')
problem=pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-2/problem 2.csv')

In [ ]:
box

In [ ]:
purchase

In [11]:
purchase.columns

Index(['PURCHASE_DATE', 'MAGIC_KEY', 'BOX_ID', 'BOX_COUNT'], dtype='object')

In [12]:
box.isna().sum()

BOX_ID             0
QUALITY            0
DELIVERY_OPTION    0
MILK               0
MEAT               0
UNIT_PRICE         0
dtype: int64

## Drop the Null Entries

In [16]:
purchase.isna().sum()

PURCHASE_DATE     0
MAGIC_KEY         0
BOX_ID           47
BOX_COUNT        47
dtype: int64

In [17]:
purchase=purchase.dropna()

In [18]:
problem.isnull().sum()

MAGIC_KEY    0
dtype: int64

## Number of Unique Magic Keys


In [19]:
l1=[]
l2=[]
for i in problem['MAGIC_KEY']:
    l1.append(i)
for i in purchase['MAGIC_KEY']:
    l2.append(i)
    
print(len(l1),len(l2))
l1=set(l1)
l2=set(l2)
print(len(l1),len(l2))
l3=l1.intersection(l2)
print(len(l3))

43518 2455817
43518 1274094
43518


<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>Merge Purchase and Box datasets</h3>
</div>


In [21]:
merged_df = pd.merge(purchase, box, on='BOX_ID')

merged_df['Date'] = pd.to_datetime(merged_df['PURCHASE_DATE'], format='%d/%m/%Y')

# Extract only the date part (day, month, and year)
#merged_df['Date'] = merged_df['Date'].dt.date


In [22]:
merged_df.columns

Index(['PURCHASE_DATE', 'MAGIC_KEY', 'BOX_ID', 'BOX_COUNT', 'QUALITY',
       'DELIVERY_OPTION', 'MILK', 'MEAT', 'UNIT_PRICE', 'Date'],
      dtype='object')

In [23]:
merged_df['cost']=merged_df['BOX_COUNT']*merged_df['UNIT_PRICE']

In [24]:
merged_df

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,Date,cost
0,1/2/2019,2CED678A247,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98,2019-02-01,12.98
1,1/2/2019,2BF58D91BA1,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98,2019-02-01,12.98
2,1/2/2019,2C15B86534E,99.0,1.0,Premium,Delivery from Collection Point,0.0,3.3,13.96,2019-02-01,13.96
3,1/2/2019,2C32D9A859A,6.0,1.0,Premium,Home Delivery - CoD,0.0,2.7,11.96,2019-02-01,11.96
4,1/2/2019,2C7A55404D1,4.0,1.0,Premium,Home Delivery - CoD,0.0,2.5,11.96,2019-02-01,11.96
...,...,...,...,...,...,...,...,...,...,...,...
2455795,28/10/2018,2BD992B5538,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98,2018-10-28,12.98
2455796,28/10/2018,2C97CD72233,17.0,1.0,Premium,Home Delivery - CoD,10.0,1.8,12.98,2018-10-28,12.98
2455797,28/10/2018,2C91C61D372,40.0,1.0,Premium,Home Delivery - CoD,12.0,1.8,19.98,2018-10-28,19.98
2455798,28/10/2018,2CD70CFC4E3,51.0,1.0,Premium,Home Delivery - CoD,18.0,2.9,23.98,2018-10-28,23.98


<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>Fearure Engineering</h3>
</div>


In [25]:
meat_avg=merged_df['MEAT'].mean()
milk_avg=merged_df['MILK'].mean()
price_avg=merged_df['cost'].mean()


In [26]:
merged_df['month'] = merged_df['Date'].dt.month
merged_df['day'] = merged_df['Date'].dt.day

In [27]:
merged_df['month'].unique()

array([ 2,  1, 12, 11, 10], dtype=int32)

In [28]:
merged_df['day']=merged_df['day'].apply(lambda x: 0 if x>15 else 1)
merged_df['MILK']=merged_df['MILK'].apply(lambda x: 1 if x>milk_avg else 0)
merged_df['MEAT']=merged_df['MEAT'].apply(lambda x: 1 if x>meat_avg else 0)
merged_df['cost']=merged_df['cost'].apply(lambda x: 1 if x>price_avg else 0)
merged_df['month']=merged_df['month'].map({1:1,2:2,10:0,11:0,12:0})

In [29]:
merged_df['sum']=merged_df['month']+merged_df['day']+merged_df['MEAT']

In [30]:
merged_df

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,Date,cost,month,day,sum
0,1/2/2019,2CED678A247,12.0,1.0,Premium,Home Delivery - CoD,0,0,12.98,2019-02-01,0,2,1,3
1,1/2/2019,2BF58D91BA1,12.0,1.0,Premium,Home Delivery - CoD,0,0,12.98,2019-02-01,0,2,1,3
2,1/2/2019,2C15B86534E,99.0,1.0,Premium,Delivery from Collection Point,0,1,13.96,2019-02-01,0,2,1,4
3,1/2/2019,2C32D9A859A,6.0,1.0,Premium,Home Delivery - CoD,0,1,11.96,2019-02-01,0,2,1,4
4,1/2/2019,2C7A55404D1,4.0,1.0,Premium,Home Delivery - CoD,0,1,11.96,2019-02-01,0,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455795,28/10/2018,2BD992B5538,12.0,1.0,Premium,Home Delivery - CoD,0,0,12.98,2018-10-28,0,0,0,0
2455796,28/10/2018,2C97CD72233,17.0,1.0,Premium,Home Delivery - CoD,1,0,12.98,2018-10-28,0,0,0,0
2455797,28/10/2018,2C91C61D372,40.0,1.0,Premium,Home Delivery - CoD,1,0,19.98,2018-10-28,1,0,0,0
2455798,28/10/2018,2CD70CFC4E3,51.0,1.0,Premium,Home Delivery - CoD,1,1,23.98,2018-10-28,1,0,0,1


In [31]:
# Assuming 'purchase' is your DataFrame and 'l1' is your list of values to filter

# Filter the DataFrame based on values in the 'MAGIC_KEY' column
filtered_df = merged_df[merged_df['MAGIC_KEY'].isin(l1)]

# Reset the index of the filtered DataFrame
filtered_df = filtered_df.reset_index(drop=True)
sorted_df = filtered_df.groupby('MAGIC_KEY').apply(lambda x: x.sort_values('Date', ascending=False)).reset_index(drop=True)



/tmp/ipykernel_34/79882275.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = filtered_df.groupby('MAGIC_KEY').apply(lambda x: x.sort_values('Date', ascending=False)).reset_index(drop=True)


In [32]:
def find_prob(df):
    data_series = df['BOX_ID']
    unique_elements = data_series.unique()
    frequency = data_series.value_counts().to_dict()
    
    score_list = [0.4 / (index + 1) + 0.1* frequency[value] for index, value in enumerate(unique_elements)]
    
    max_index = score_list.index(max(score_list))
    
    return int(unique_elements[max_index])
def find_std(df):
    df['dis']=df['Date'].diff().dt.days
    df=df.dropna()
    if df.dis.count()==1:
        return .98
    x=df['dis'].std()
    return x
def find_mean(df):
    df['dis']=df['Date'].diff().dt.days
    df=df.dropna()
    x=df['dis'].mean()
    return x
cur_date = pd.to_datetime('10/03/2019', format='%d/%m/%Y').date()
print(cur_date)
def find_dis(df):
    df=df.reset_index(drop=True)
    index=df.tail(1).index[0]
    date=df['Date'][index]
    print(date)
    diff=cur_date-date
    print(diff)

def find_prob3(df):
    # Count frequency of BOX_ID and compute score_list
    frequency = df['BOX_ID'].value_counts()
    score_list = df['sum'] + 0.08 * frequency[df['BOX_ID']].values
    
    # Find the index of the maximum value in score_list
    max_index = score_list.idxmax()
    
    # Return the corresponding BOX_ID
    return int(df.loc[max_index, 'BOX_ID'])

    
def find_last(df):
    index=df.head(1).index[0]
    box=int(df['BOX_ID'][index])
    return box
    

2019-03-10


In [34]:
# grouped_df

In [35]:

final_file=sorted_df.groupby('MAGIC_KEY').apply(find_prob).reset_index()
final_file.columns = ['MAGIC_KEY', 'BOX_ID']

/tmp/ipykernel_34/279258853.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_file=sorted_df.groupby('MAGIC_KEY').apply(find_prob).reset_index()


<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>Submission File</h3>
</div>


In [36]:
output=pd.merge(problem,final_file,on='MAGIC_KEY')
output

,MAGIC_KEY,BOX_ID
0,2C0B5F97180,141
1,2C2F659EF00,144
2,29D629EDF20,237
3,2A7073E2FF0,230
4,2A9B9BE1210,270
...,...,...
43513,2C49A3D7EAD,134
43514,2C49C75F0D8,182
43515,2C49CA2A632,108
43516,2C49D52753F,42


In [37]:
output.to_csv('submission.csv',index=False)